# Optimized Cellpose Installation for MyBinder

This notebook provides a robust installation process for Cellpose in MyBinder environments.

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/your-username/your-repo/HEAD?filepath=cellpose_optimized.ipynb)

## Key Improvements:
- Simplified installation process
- Better error handling
- Resource-efficient approach
- Timeout management
- Fallback options

## Step 1: Environment Check and Setup

In [ ]:
import sys
import os
import subprocess
import time
from pathlib import Path

print(f"Python version: {sys.version}")
print(f"Current working directory: {os.getcwd()}")
print(f"Available memory: {os.sysconf('SC_PAGE_SIZE') * os.sysconf('SC_PHYS_PAGES') / (1024.**3):.2f} GB")

# Check existing environment
def run_command(cmd, timeout=60):
    """Run command with timeout and proper error handling"""
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=timeout)
        return result.returncode == 0, result.stdout, result.stderr
    except subprocess.TimeoutExpired:
        return False, "", f"Command timed out after {timeout} seconds"
    except Exception as e:
        return False, "", str(e)

# Check for conda/mamba
success, stdout, stderr = run_command("which conda")
if success:
    CONDA_CMD = "conda"
    print(f"✅ Found conda at: {stdout.strip()}")
else:
    success, stdout, stderr = run_command("which mamba")
    if success:
        CONDA_CMD = "mamba"
        print(f"✅ Found mamba at: {stdout.strip()}")
    else:
        CONDA_CMD = None
        print("❌ Neither conda nor mamba found")

## Step 2: Install Miniforge (if needed)

In [ ]:
if CONDA_CMD is None:
    print("Installing miniforge3...")
    
    # Download miniforge
    download_cmd = "wget -q https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh -O miniforge3.sh"
    success, stdout, stderr = run_command(download_cmd, timeout=180)
    
    if success:
        print("✅ Downloaded miniforge3")
        
        # Install miniforge
        install_cmd = "bash miniforge3.sh -b -p $HOME/miniforge3"
        success, stdout, stderr = run_command(install_cmd, timeout=300)
        
        if success:
            # Update PATH
            miniforge_path = os.path.expanduser("~/miniforge3/bin")
            os.environ['PATH'] = f"{miniforge_path}:{os.environ['PATH']}"
            CONDA_CMD = f"{miniforge_path}/mamba"
            print(f"✅ Installed miniforge3, using: {CONDA_CMD}")
        else:
            print(f"❌ Failed to install miniforge3: {stderr}")
            CONDA_CMD = None
    else:
        print(f"❌ Failed to download miniforge3: {stderr}")
        CONDA_CMD = None
else:
    print(f"Using existing {CONDA_CMD}")

# Verify conda/mamba works
if CONDA_CMD:
    success, stdout, stderr = run_command(f"{CONDA_CMD} --version")
    if success:
        print(f"✅ {CONDA_CMD} version: {stdout.strip()}")
    else:
        print(f"❌ {CONDA_CMD} not working: {stderr}")
        CONDA_CMD = None

## Step 3: Lightweight Installation Approach

In [ ]:
# Try direct pip installation first (faster for MyBinder)
def install_cellpose_direct():
    """Try installing cellpose directly with pip"""
    print("Attempting direct pip installation...")
    
    # Install basic dependencies first
    basic_deps = [
        "numpy", "scipy", "matplotlib", "scikit-image", 
        "opencv-python-headless", "tifffile", "torch", "torchvision"
    ]
    
    for dep in basic_deps:
        print(f"Installing {dep}...")
        success, stdout, stderr = run_command(f"pip install {dep} --quiet", timeout=180)
        if success:
            print(f"  ✅ {dep} installed")
        else:
            print(f"  ⚠️ {dep} failed: {stderr[:100]}...")
    
    # Install cellpose
    print("Installing cellpose...")
    success, stdout, stderr = run_command("pip install cellpose --quiet", timeout=300)
    
    if success:
        print("✅ Cellpose installed successfully!")
        return True
    else:
        print(f"❌ Cellpose installation failed: {stderr[:200]}...")
        return False

# Try direct installation
direct_success = install_cellpose_direct()

## Step 4: Conda Installation (Fallback)

In [ ]:
if not direct_success and CONDA_CMD:
    print("Direct installation failed, trying conda approach...")
    
    env_name = "cellpose_env"
    
    # Create environment with essential packages only
    print(f"Creating minimal environment: {env_name}")
    create_cmd = f"{CONDA_CMD} create -n {env_name} python=3.9 pip -y"
    success, stdout, stderr = run_command(create_cmd, timeout=300)
    
    if success:
        print(f"✅ Environment {env_name} created")
        
        # Install core dependencies via conda
        print("Installing core dependencies...")
        deps_cmd = f"{CONDA_CMD} install -n {env_name} -c conda-forge numpy matplotlib scikit-image opencv -y"
        success, stdout, stderr = run_command(deps_cmd, timeout=300)
        
        if success:
            print("✅ Core dependencies installed")
            
            # Install cellpose via pip in environment
            print("Installing cellpose in environment...")
            cellpose_cmd = f"{CONDA_CMD} run -n {env_name} pip install cellpose"
            success, stdout, stderr = run_command(cellpose_cmd, timeout=300)
            
            if success:
                print("✅ Cellpose installed in conda environment!")
                CONDA_ENV = env_name
            else:
                print(f"❌ Cellpose conda installation failed: {stderr[:200]}...")
                CONDA_ENV = None
        else:
            print(f"❌ Dependencies installation failed: {stderr[:200]}...")
            CONDA_ENV = None
    else:
        print(f"❌ Environment creation failed: {stderr[:200]}...")
        CONDA_ENV = None
else:
    CONDA_ENV = None
    if direct_success:
        print("Using direct installation")
    else:
        print("No conda available for fallback")

## Step 5: Verify Installation

In [ ]:
def test_cellpose_installation(use_conda_env=False):
    """Test cellpose installation"""
    test_script = '''
try:
    import cellpose
    from cellpose import models
    import numpy as np
    
    print(f"✅ Cellpose version: {cellpose.__version__}")
    
    # Test model loading (without downloading)
    try:
        model = models.Cellpose(gpu=False, model_type='cyto')
        print("✅ Model initialization successful")
    except Exception as e:
        print(f"⚠️ Model loading issue: {e}")
    
    print(f"Available models: {models.MODEL_NAMES}")
    print("✅ Cellpose installation test PASSED")
    
except ImportError as e:
    print(f"❌ Import error: {e}")
except Exception as e:
    print(f"❌ Unexpected error: {e}")
'''
    
    with open('test_cellpose.py', 'w') as f:
        f.write(test_script)
    
    if use_conda_env and CONDA_ENV:
        cmd = f"{CONDA_CMD} run -n {CONDA_ENV} python test_cellpose.py"
    else:
        cmd = "python test_cellpose.py"
    
    success, stdout, stderr = run_command(cmd, timeout=60)
    
    print("Installation Test Results:")
    print("-" * 40)
    if stdout:
        print(stdout)
    if stderr and "warning" not in stderr.lower():
        print(f"Errors: {stderr}")
    
    return "✅ Cellpose installation test PASSED" in stdout

# Test the installation
if direct_success:
    installation_ok = test_cellpose_installation(use_conda_env=False)
else:
    installation_ok = test_cellpose_installation(use_conda_env=True)

print(f"\nFinal Status: {'✅ SUCCESS' if installation_ok else '❌ FAILED'}")

## Step 6: Create Test Data

In [ ]:
# Only proceed if installation was successful
if installation_ok:
    print("Creating synthetic test data...")
    
    create_data_script = '''
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
import tifffile

def create_test_cells(size=(256, 256), n_cells=10):
    """Create simple synthetic cell data"""
    img = np.zeros(size, dtype=np.float32)
    
    for i in range(n_cells):
        # Random position
        y = np.random.randint(30, size[0]-30)
        x = np.random.randint(30, size[1]-30)
        
        # Random size
        radius = np.random.randint(8, 20)
        
        # Create cell
        yy, xx = np.ogrid[:size[0], :size[1]]
        mask = (xx - x)**2 + (yy - y)**2 <= radius**2
        img[mask] = np.random.randint(150, 255)
    
    # Add noise and smooth
    img += np.random.normal(0, 5, size)
    img = ndimage.gaussian_filter(img, sigma=0.5)
    img = np.clip(img, 0, 255).astype(np.uint8)
    
    return img

# Generate test image
test_img = create_test_cells()
tifffile.imwrite('test_cells.tif', test_img)

# Create preview
plt.figure(figsize=(6, 6))
plt.imshow(test_img, cmap='gray')
plt.title('Test Cells')
plt.axis('off')
plt.savefig('test_preview.png', dpi=100, bbox_inches='tight')
plt.show()

print(f"Test image created: {test_img.shape}, range: {test_img.min()}-{test_img.max()}")
'''
    
    with open('create_data.py', 'w') as f:
        f.write(create_data_script)
    
    if CONDA_ENV:
        cmd = f"{CONDA_CMD} run -n {CONDA_ENV} python create_data.py"
    else:
        cmd = "python create_data.py"
    
    success, stdout, stderr = run_command(cmd, timeout=60)
    
    if success:
        print("✅ Test data created successfully")
        if stdout:
            print(stdout)
    else:
        print(f"❌ Failed to create test data: {stderr}")
else:
    print("⚠️ Skipping test data creation due to installation issues")

## Step 7: Run Cellpose Segmentation

In [ ]:
if installation_ok and os.path.exists('test_cells.tif'):
    print("Running Cellpose segmentation...")
    
    segmentation_script = '''
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models
import tifffile
import time

# Load test image
img = tifffile.imread('test_cells.tif')
print(f"Loaded image: {img.shape}")

# Initialize model
print("Initializing Cellpose model...")
model = models.Cellpose(gpu=False, model_type='cyto')

# Run segmentation
print("Running segmentation...")
start_time = time.time()

masks, flows, styles, diams = model.eval(
    img, 
    diameter=None, 
    channels=[0,0],
    flow_threshold=0.4,
    cellprob_threshold=0.0
)

end_time = time.time()
n_cells = len(np.unique(masks)) - 1

print(f"Segmentation completed in {end_time - start_time:.2f} seconds")
print(f"Detected {n_cells} cells")
print(f"Estimated diameter: {diams:.1f} pixels")

# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Original
axes[0].imshow(img, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

# Segmentation
axes[1].imshow(masks, cmap='nipy_spectral')
axes[1].set_title(f'Segmentation ({n_cells} cells)')
axes[1].axis('off')

plt.tight_layout()
plt.savefig('segmentation_result.png', dpi=100, bbox_inches='tight')
plt.show()

# Save results
tifffile.imwrite('segmentation_masks.tif', masks.astype(np.uint16))
print("Results saved to segmentation_masks.tif")
print("✅ Segmentation test completed successfully!")
'''
    
    with open('run_segmentation.py', 'w') as f:
        f.write(segmentation_script)
    
    if CONDA_ENV:
        cmd = f"{CONDA_CMD} run -n {CONDA_ENV} python run_segmentation.py"
    else:
        cmd = "python run_segmentation.py"
    
    success, stdout, stderr = run_command(cmd, timeout=120)
    
    if success:
        print("✅ Segmentation completed successfully!")
        if stdout:
            print(stdout)
    else:
        print(f"❌ Segmentation failed: {stderr}")
else:
    print("⚠️ Skipping segmentation test due to previous issues")

## Step 8: Usage Examples and Summary

In [ ]:
print("📋 CELLPOSE MYBINDER SETUP SUMMARY")
print("=" * 50)

if installation_ok:
    print("✅ Status: SUCCESSFUL INSTALLATION")
    print("\n🔧 Usage Examples:")
    print("""
    # Basic usage:
    from cellpose import models
    import tifffile
    
    # Load your image
    img = tifffile.imread('your_image.tif')
    
    # Initialize model
    model = models.Cellpose(gpu=False, model_type='cyto')
    
    # Run segmentation
    masks, flows, styles, diams = model.eval(
        img, 
        diameter=None, 
        channels=[0,0]  # grayscale
    )
    
    # Save results
    tifffile.imwrite('results.tif', masks)
    """)
    
    print("\n📁 Generated Files:")
    files_to_check = [
        'test_cells.tif', 'test_preview.png', 
        'segmentation_result.png', 'segmentation_masks.tif'
    ]
    
    for file in files_to_check:
        if os.path.exists(file):
            size = os.path.getsize(file) / 1024
            print(f"  ✅ {file} ({size:.1f} KB)")
        else:
            print(f"  ❌ {file} (not found)")
    
    print("\n🎯 Ready for your research!")
    print("You can now upload your own images and start segmenting.")
    
else:
    print("❌ Status: INSTALLATION FAILED")
    print("\n🔧 Troubleshooting:")
    print("1. Try restarting the kernel and running cells again")
    print("2. Check MyBinder resource limits")
    print("3. Consider using a simpler installation approach")
    print("4. Contact course instructors for support")

print("\n" + "=" * 50)

## 💡 Tips for MyBinder Success

**If you continue to experience issues:**

1. **Restart and Retry**: Restart the kernel and run cells sequentially
2. **Resource Management**: MyBinder has limited resources - avoid running multiple notebooks simultaneously
3. **Simplified Approach**: Comment out conda installation and rely only on direct pip installation
4. **Alternative Environment**: Consider using Google Colab or local Jupyter installation
5. **Pre-built Environment**: Create a custom Binder repository with pre-installed dependencies

**For Course Instructors:**

Create an `environment.yml` file in your repository:

```yaml
name: cellpose-course
channels:
  - conda-forge
  - defaults
dependencies:
  - python=3.9
  - numpy
  - matplotlib
  - scikit-image
  - opencv
  - tifffile
  - pip
  - pip:
    - cellpose
```

This will pre-install dependencies when MyBinder builds the environment, avoiding installation timeouts during notebook execution.